# Gather and Merge "Ordenes de Trabajo"
Date: 01/07/2024
Author: Ernesto Palacios
epalacios@eerssa.gob.ec

Development book:

https://ipywidgets.readthedocs.io/en/stable/

In [1]:
# Librerias requeridas:

%pip install --upgrade pymupdf
%pip install PyPDF2
%pip install pandas


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   - -------------------------------------- 0.1/3.2 MB 2.4 MB/s eta 0:00:02
   --- ------------------------------------ 0.3/3.2 MB 3.3 MB/s eta 0:00:01
   ------- -------------------------------- 0.6/3.2 MB 4.4 MB/s eta 0:00:01
   ----------- ---------------------------- 1.0/3.2 MB 5.5 MB/s eta 0:00:01
   --------------- ------------------------ 1.3/3.2 MB 5.1 MB/s eta 0:00:01
   ----------------------- ---------------- 1.9/3.2 MB 6.2 MB/s eta 0:00:01
   -------------------------- ------------- 2.1/3.2 MB 6.1 MB/s eta 0:00:01
   ----------------------------- ---------- 2.4/3.2 MB 6.0 MB/s eta 0:00:01
   --------------------------------- ------ 2.7/3.2 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------  3.2/3.2 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------  3.2/3.2 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------  3.2/3.2 MB 6.4 MB/s eta 0:00:01
   ---------------


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Import Libraries
import os
import logging
from pprint import pprint
from PyPDF2 import PdfWriter, PdfReader, PdfFileMerger

import pandas as pd

logging.basicConfig(level=logging.DEBUG)

# Define a function that takes the name of the folder and outputs 
# what the resulting PDF file should be named.

def getPDFName( current_folder ):
    try:
        data = current_folder.split()
        position = data[0]
        cuadrilla = data [1] # not used
        nombre = " ".join(data[2:]).title()
    except:
        print("No se ha podido extraer el nombre de Cuadrilla")
        position = '00'
        nombre = 'Sin Nombre'

    respuesta = " ".join( [position, nombre] )  
    
    return respuesta 

# Array with the names of the PDF files inside each subDirectory (Cuadrilla)

def getPDFListNames( root, path ):
    try:
        abs_path = os.path.join( root, path )
        pdf_files = [f for f in os.listdir(abs_path) if f.endswith('.pdf')] # Gemini
        path_files = [ os.path.join( abs_path, item ) for item in pdf_files ]
    except:
        print("No es un directorio válido")
        return []
    
    return path_files    


# create & open the resuling file
# Open the current file
# Check the number of pages of the file
# Strip the first page, and save the ohther pages to the file

def unirPDFs( listado, nombrePDFUnido):
    try:
        merger = PdfWriter()
        for pdf in listado:
            pdf_reader = PdfReader(pdf)
            num_pages = len(pdf_reader.pages)
            
            if num_pages > 2:
                merger.append(pdf, [1,2])
            else:
                merger.append(pdf,[1])

        merger.write( nombrePDFUnido )
        merger.close()
        success = True
    except:
        success = False
    return success


In [3]:
# https://medium.com/@mgkyawzayya/splitting-and-merging-pdf-files-with-python-using-pypdf2-cfce5c948c36


#root_dir = "F:\\INFORMES\\EERSSA\\ACTIVIDADES DIARIAS DE TRABAJO DE LAS CUADRILLAS\\2024\\04 ABRIL"
root_dir = "D:\\INFORMES\\EERSSA\\ACTIVIDADES DIARIAS DE TRABAJO DE LAS CUADRILLAS\\2024\\09 SEPTIEMBRE"

try:
    nn_mesActual, mes_actual = os.path.basename(root_dir).split()
    prefijo_nombrePDF = nn_mesActual + " " + mes_actual.lower() + " - "
    prefijo_nombrePDF = os.path.join( root_dir, prefijo_nombrePDF ) # Aboslute/Relative Path
except: # Place holder in case diffent directory name
    nn_mesActual = '00'  
    mes_actual = 'NombreMes'
    

try:
    folders = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
except:
    print("El directorio no es válido.")
    
pprint(folders)
# Al final de esta operación espero obtener una lista con las carpetas de las Cuadrillas.

['02 CUADRILLA ZAMORA',
 '03 CUADRILLA ALUMBRADO ZAMORA',
 '04 CUADRILLA YACUAMBI',
 '05 CUADRILLA YANTZAZA',
 '06 CUADRILLA ENERGIZADA YANTZAZA',
 '07 CUADRILLA GUAYZIMI',
 '08 CUADRILLA EL PANGUI',
 '09 CUADRILLA GUALAQUIZA',
 '21 (electricistas) Ag. Zamora',
 '22 (electricistas) Ag. Yantzaza',
 '23 (electricistas) Ag. El Pangui',
 '24 (electricistas) Ag. Gualaquiza']


In [4]:
# Crear un Pandas.Dataframe organizando la información:

df = pd.DataFrame( folders, columns = ["carpetas"])
df["nombre_pdf_unificado"] = prefijo_nombrePDF + df["carpetas"].apply( lambda x: getPDFName(x) ) + ".pdf"
df["listaPDFs"] = df["carpetas"].apply( lambda x: getPDFListNames( root_dir, x ) )
df["estadoMergePDF"] = df.apply( lambda row: unirPDFs( row['listaPDFs'],row['nombre_pdf_unificado']), axis = 1)
df

,carpetas,nombre_pdf_unificado,listaPDFs,estadoMergePDF
0,02 CUADRILLA ZAMORA,D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRAB...,[D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRA...,True
1,03 CUADRILLA ALUMBRADO ZAMORA,D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRAB...,[D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRA...,True
2,04 CUADRILLA YACUAMBI,D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRAB...,[D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRA...,True
3,05 CUADRILLA YANTZAZA,D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRAB...,[D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRA...,True
4,06 CUADRILLA ENERGIZADA YANTZAZA,D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRAB...,[D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRA...,True
5,07 CUADRILLA GUAYZIMI,D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRAB...,[D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRA...,True
6,08 CUADRILLA EL PANGUI,D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRAB...,[D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRA...,True
7,09 CUADRILLA GUALAQUIZA,D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRAB...,[D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRA...,True
8,21 (electricistas) Ag. Zamora,D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRAB...,[D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRA...,True
9,22 (electricistas) Ag. Yantzaza,D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRAB...,[D:\INFORMES\EERSSA\ACTIVIDADES DIARIAS DE TRA...,True



-----

Develop

## Exploring the os library

In [25]:
import os 

anio_dir = "C:\\Users\\epalacios\\OneDrive\\01 JEZO\\01 ACTIVIDADES DIARIAS DE TRABAJO DE LAS AGENCIAS\\2024\\01 ENERO"

def get_directory_depth(start_path=anio_dir):
    depth = 0
    for root, dirs, files in os.walk(start_path):
        depth = max(depth, root.count(os.sep) - start_path.count(os.sep))
    return depth

depth = get_directory_depth()
print("Depth of current directory:", depth)

Depth of current directory: 1
